REFERENCES:  https://www.geeksforgeeks.org/implementing-pca-in-python-with-scikit-learn/
,

https://towardsdatascience.com/math-neural-network-from-scratch-in-python-d6da9f29ce65 ,
https://keras.io/guides/training_with_built_in_methods/

In [1]:
import time
import pandas as pd
import numpy as np
import h5py
import random
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from sklearn.preprocessing import MinMaxScaler
from itertools import islice
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.activations import softmax
import gc
import itertools
from sklearn.metrics import mean_squared_error
import sklearn
from numpy import genfromtxt
from os import listdir
from os.path import isfile, join
import os
import copy
from scipy import interpolate
from scipy.spatial import distance
import time
import numpy as np

#np.set_printoptions(suppress=True)


def load_dataset(datasets,shape,labels):
    start = time.time()
    count = 1
    wavelength_list = []
    intensity_list = []
    label_list = []
    get_wavelength = True
    for dataset_name in datasets:
        with h5py.File(dataset_name, "r") as f:
            if get_wavelength: wavelengths = np.array(np.split(f['wavelength'][:], f['wavelength'][:].shape[0]//shape))
            intensities = np.array(np.split(f['intensity'][:], f['intensity'][:].shape[0]//shape))
            labels = np.array(np.split(f['label'][:], f['label'][:].shape[0]//labels))/100
            if get_wavelength: wavelength_list.append(wavelengths)
            intensity_list.append(intensities)
            label_list.append(labels)
            print(dataset_name, " Done")
        get_wavelength = False
    wavelengths = np.array(list(itertools.chain.from_iterable(wavelength_list)))
    intensities = np.array(list(itertools.chain.from_iterable(intensity_list)))
    labels = np.array(list(itertools.chain.from_iterable(label_list)))
    print(round(time.time()-start,2), "seconds")    
    return wavelengths, intensities, labels



In [2]:
datasets_to_load = ['dataset7.h5']
wavelengths_list, c, labels_list = load_dataset(datasets_to_load,4094,24)
current_elements = ['C', 'Mg', 'Al', 'Si', 'Ti', 'V', 'Fe', 'Co', 'Ni', 'Cu', 'Mn', 'Li', 'Zn', 'O', 'P', 'Pt', 'Au', 'Ag', 'Ir', 'W', 'Pd', 'Sn', 'Tc', 'Y']


dataset7.h5  Done
44.72 seconds


In [3]:
#current_elements.index('O')

In [4]:
df_wavelengths=pd.DataFrame(wavelengths_list)
df_wavelengths.shape


(53660, 4094)

In [5]:
df_wavelengths.head(5)

0          1          2          3          4          5          6     \
0  184.9409  185.00817  185.07544  185.14271  185.20998  185.27725  185.34452   
1  184.9409  185.00817  185.07544  185.14271  185.20998  185.27725  185.34452   
2  184.9409  185.00817  185.07544  185.14271  185.20998  185.27725  185.34452   
3  184.9409  185.00817  185.07544  185.14271  185.20998  185.27725  185.34452   
4  184.9409  185.00817  185.07544  185.14271  185.20998  185.27725  185.34452   

        7          8          9     ...       4084       4085       4086  \
0  185.41179  185.47906  185.54633  ...  459.67158  459.73885  459.80612   
1  185.41179  185.47906  185.54633  ...  459.67158  459.73885  459.80612   
2  185.41179  185.47906  185.54633  ...  459.67158  459.73885  459.80612   
3  185.41179  185.47906  185.54633  ...  459.67158  459.73885  459.80612   
4  185.41179  185.47906  185.54633  ...  459.67158  459.73885  459.80612   

        4087       4088       4089      4090       4091       4092       4093  
0  459.87339  459.94066  460.00793  460.0752  460.14247  460.20974  460.27701  
1  459.87339  459.94066  460.00793  460.0752  460.14247  460.20974  460.27701  
2  459.87339  459.94066  460.00793  460.0752  460.14247  460.20974  460.27701  
3  459.87339  459.94066  460.00793  460.0752  460.14247  460.20974  460.27701  
4  459.87339  459.94066  460.00793  460.0752  460.14247  460.20974  460.27701  

[5 rows x 4094 columns]

In [6]:
df_intensity=pd.DataFrame(c)
df_intensity.shape

(53660, 4094)

In [7]:
df_intensity.head()

0             1              2             3             4     \
0  0.000000e+00  2.000000e-10       0.000003  6.425709e-04  1.575615e-03   
1  9.886173e-54  1.163524e-55       0.000000  1.554174e-51  0.000000e+00   
2  0.000000e+00  0.000000e+00       0.000000  0.000000e+00  5.000000e-10   
3  0.000000e+00  4.240042e+04  271401.081935  2.087370e+04  2.197288e+04   
4  2.212317e+00  2.136087e+00       2.061629  1.988922e+00  1.917944e+00   

           5             6            7             8             9     ...  \
0      0.000046  1.930000e-08     0.000000      0.000000      0.000000  ...   
1      0.000000  0.000000e+00     0.000000      0.000000      0.000000  ...   
2      0.000258  5.542503e+00  2288.572593  11510.308254  13851.330353  ...   
3  34653.560832  2.914393e+03    49.993821      0.013321      0.000000  ...   
4      1.848676  1.781095e+00     1.715182      1.650916      1.588275  ...   

   4084  4085  4086  4087  4088  4089  4090  4091  4092  4093  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

[5 rows x 4094 columns]

In [8]:
df_labels_list=pd.DataFrame(labels_list)
df_labels_list.head()

0    1    2    3    4    5    6    7    8    9   ...   14   15   16   17  \
0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
1  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
2  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
3  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
4  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   

    18   19   20   21   22   23  
0  0.0  0.0  0.0  0.0  0.0  0.0  
1  0.0  0.0  0.0  0.0  0.0  0.0  
2  0.0  0.0  0.0  0.0  0.0  0.0  
3  0.0  0.0  0.0  0.0  0.0  0.0  
4  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 24 columns]

In [9]:
df_labels_list.shape

(53660, 24)

In [10]:
df_labels_list[13]!=0.0

0        False
1        False
2        False
3        False
4        False
         ...  
53655    False
53656    False
53657    False
53658    False
53659    False
Name: 13, Length: 53660, dtype: bool

In [11]:
#row 13 is 'O' which is gas element so droping all the rows which has oxygen
#get rows with gas elements .i.e. rows which has value other than 0 in column 13
rows_gas = df_labels_list[(df_labels_list[[13]] != 0.0).all(axis=1)]
rows_gas

0     1     2     3    4    5    6    7    8    9   ...   14   15  \
13     0.00  0.00  0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
36     0.99  0.00  0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
58     0.00  0.99  0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
79     0.00  0.00  0.99  0.00  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
99     0.00  0.00  0.00  0.99  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
...     ...   ...   ...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
53535  0.00  0.00  0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
53536  0.00  0.00  0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
53537  0.00  0.00  0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
53538  0.00  0.00  0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
53539  0.00  0.00  0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   

        16   17   18   19    20    21    22   23  
13     0.0  0.0  0.0  0.0  0.00  0.00  0.00  0.0  
36     0.0  0.0  0.0  0.0  0.00  0.00  0.00  0.0  
58     0.0  0.0  0.0  0.0  0.00  0.00  0.00  0.0  
79     0.0  0.0  0.0  0.0  0.00  0.00  0.00  0.0  
99     0.0  0.0  0.0  0.0  0.00  0.00  0.00  0.0  
...    ...  ...  ...  ...   ...   ...   ...  ...  
53535  0.0  0.0  0.0  0.0  0.15  0.00  0.30  0.0  
53536  0.0  0.0  0.0  0.0  0.15  0.00  0.00  0.3  
53537  0.0  0.0  0.0  0.0  0.00  0.15  0.30  0.0  
53538  0.0  0.0  0.0  0.0  0.00  0.15  0.00  0.3  
53539  0.0  0.0  0.0  0.0  0.00  0.00  0.15  0.3  

[5567 rows x 24 columns]

In [12]:
rows_of_gas=rows_gas.index
rows_of_gas

Int64Index([   13,    36,    58,    79,    99,   118,   136,   153,   169,
              184,
            ...
            53530, 53531, 53532, 53533, 53534, 53535, 53536, 53537, 53538,
            53539],
           dtype='int64', length=5567)

In [13]:

df_intensity=df_intensity.drop(index=rows_of_gas)
df_intensity.shape

(48093, 4094)

In [14]:
df_wavelengths=df_wavelengths.drop(index=rows_of_gas)
df_wavelengths.shape

(48093, 4094)

In [15]:
df_labels_list=df_labels_list.drop(index=rows_of_gas)
df_labels_list.shape



(48093, 24)

In [16]:
df_labels_list=df_labels_list.drop(13, axis=1)
df_labels_list.shape

(48093, 23)

## split dataset for train set and test set

In [17]:
x=df_intensity
y=df_labels_list

In [ ]:
# Split x into train and test sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val= train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [ ]:
# Print the shape of the resulting data sets
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

X_train shape: (34626, 4094)
y_train shape: (34626, 23)
X_test shape: (4810, 4094)
y_test shape: (4810, 23)
X_val shape: (8657, 4094)
y_val shape: (8657, 23)


In [ ]:
model = Sequential()
model.add(Dense(2000, batch_input_shape=(None,4094),activation='relu'))
model.add(Dense(1000, batch_input_shape=(None,4094),activation='relu'))
model.add(Dense(100, batch_input_shape=(None,4094),activation='relu'))
model.add(Dense(64, batch_input_shape=(None,4094),activation='relu'))
model.add(Dense(23, activation=softmax))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=200, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

# Make predictions
#predictions = model.predict(X_new)

2023-03-10 12:55:36.376950: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 12:55:37.525259: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22352 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:41:00.0, compute capability: 8.6


RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [38]:
#run_1, epoch=100
model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 2000)              8190000   
                                                                 
 dense_12 (Dense)            (None, 1000)              2001000   
                                                                 
 dense_13 (Dense)            (None, 100)               100100    
                                                                 
 dense_14 (Dense)            (None, 64)                6464      
                                                                 
 dense_15 (Dense)            (None, 23)                1495      
                                                                 
Total params: 10,299,059
Trainable params: 10,299,059
Non-trainable params: 0
_________________________________________________________________


In [40]:
#run_1,epoch=100
model.evaluate(X_val, y_val)

271/271 [==============================] - 1s 3ms/step - loss: 3.1358 - accuracy: 0.0399


[3.135817289352417, 0.039852142333984375]

In [44]:
#run_2,epoch=200
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 2000)              8190000   
                                                                 
 dense_17 (Dense)            (None, 1000)              2001000   
                                                                 
 dense_18 (Dense)            (None, 100)               100100    
                                                                 
 dense_19 (Dense)            (None, 64)                6464      
                                                                 
 dense_20 (Dense)            (None, 23)                1495      
                                                                 
Total params: 10,299,059
Trainable params: 10,299,059
Non-trainable params: 0
_________________________________________________________________


In [45]:
#run_2,epoch=00
model.evaluate(X_val, y_val)

271/271 [==============================] - 1s 2ms/step - loss: 3.1359 - accuracy: 0.0447


[3.135901689529419, 0.044703707098960876]

In [23]:
accuracy

0.03846153989434242

In [41]:
predictions = model.predict(X_val)

271/271 [==============================] - 1s 2ms/step


In [42]:
predictions

array([[0.04338201, 0.04357356, 0.04209119, ..., 0.0433329 , 0.04315581,
        0.04352909],
       [0.04338201, 0.04357356, 0.04209119, ..., 0.0433329 , 0.04315581,
        0.04352909],
       [0.04338201, 0.04357356, 0.04209119, ..., 0.0433329 , 0.04315581,
        0.04352909],
       ...,
       [0.04338201, 0.04357356, 0.04209119, ..., 0.0433329 , 0.04315581,
        0.04352909],
       [0.04338201, 0.04357356, 0.04209119, ..., 0.0433329 , 0.04315581,
        0.04352909],
       [0.04338201, 0.04357356, 0.04209119, ..., 0.0433329 , 0.04315581,
        0.04352909]], dtype=float32)

In [26]:
df_predictions = pd.DataFrame(predictions)
df_predictions

0         1         2         3         4         5         6   \
0     0.043275  0.043761  0.043767  0.043723  0.042358  0.044714  0.042812   
1     0.043275  0.043761  0.043767  0.043723  0.042358  0.044714  0.042812   
2     0.043275  0.043761  0.043767  0.043723  0.042358  0.044714  0.042812   
3     0.043275  0.043761  0.043767  0.043723  0.042358  0.044714  0.042812   
4     0.043275  0.043761  0.043767  0.043723  0.042358  0.044714  0.042812   
...        ...       ...       ...       ...       ...       ...       ...   
8652  0.043275  0.043761  0.043767  0.043723  0.042358  0.044714  0.042812   
8653  0.043275  0.043761  0.043767  0.043723  0.042358  0.044714  0.042812   
8654  0.043275  0.043761  0.043767  0.043723  0.042358  0.044714  0.042812   
8655  0.043275  0.043761  0.043767  0.043723  0.042358  0.044714  0.042812   
8656  0.043275  0.043761  0.043767  0.043723  0.042358  0.044714  0.042812   

            7         8         9   ...        13       14        15  \
0     0.042818  0.043668  0.043571  ...  0.042979  0.04372  0.043844   
1     0.042818  0.043668  0.043571  ...  0.042979  0.04372  0.043844   
2     0.042818  0.043668  0.043571  ...  0.042979  0.04372  0.043844   
3     0.042818  0.043668  0.043571  ...  0.042979  0.04372  0.043844   
4     0.042818  0.043668  0.043571  ...  0.042979  0.04372  0.043844   
...        ...       ...       ...  ...       ...      ...       ...   
8652  0.042818  0.043668  0.043571  ...  0.042979  0.04372  0.043844   
8653  0.042818  0.043668  0.043571  ...  0.042979  0.04372  0.043844   
8654  0.042818  0.043668  0.043571  ...  0.042979  0.04372  0.043844   
8655  0.042818  0.043668  0.043571  ...  0.042979  0.04372  0.043844   
8656  0.042818  0.043668  0.043571  ...  0.042979  0.04372  0.043844   

            16        17        18        19      20        21        22  
0     0.044813  0.043182  0.043908  0.043096  0.0436  0.044138  0.042862  
1     0.044813  0.043182  0.043908  0.043096  0.0436  0.044138  0.042862  
2     0.044813  0.043182  0.043908  0.043096  0.0436  0.044138  0.042862  
3     0.044813  0.043182  0.043908  0.043096  0.0436  0.044138  0.042862  
4     0.044813  0.043182  0.043908  0.043096  0.0436  0.044138  0.042862  
...        ...       ...       ...       ...     ...       ...       ...  
8652  0.044813  0.043182  0.043908  0.043096  0.0436  0.044138  0.042862  
8653  0.044813  0.043182  0.043908  0.043096  0.0436  0.044138  0.042862  
8654  0.044813  0.043182  0.043908  0.043096  0.0436  0.044138  0.042862  
8655  0.044813  0.043182  0.043908  0.043096  0.0436  0.044138  0.042862  
8656  0.044813  0.043182  0.043908  0.043096  0.0436  0.044138  0.042862  

[8657 rows x 23 columns]

In [27]:
df_predictions_original = pd.DataFrame(y_val)
df_predictions_original

0    1     2     3    4    5     6    7    8    9   ...    14   15  \
51266  0.00  0.0  0.00  0.00  0.0  0.0  0.00  0.0  0.0  0.3  ...  0.00  0.0   
42571  0.00  0.0  0.00  0.00  0.0  0.8  0.10  0.1  0.0  0.0  ...  0.00  0.0   
13520  0.00  0.0  0.00  0.00  0.0  0.0  0.00  0.0  0.0  0.0  ...  0.00  0.0   
43763  0.10  0.0  0.00  0.00  0.0  0.0  0.00  0.0  0.0  0.0  ...  0.00  0.8   
44799  0.00  0.0  0.00  0.00  0.0  0.0  0.10  0.0  0.0  0.0  ...  0.00  0.0   
...     ...  ...   ...   ...  ...  ...   ...  ...  ...  ...  ...   ...  ...   
34905  0.00  0.0  0.00  0.00  0.0  0.0  0.00  0.0  0.4  0.0  ...  0.00  0.0   
28204  0.00  0.0  0.00  0.33  0.0  0.0  0.00  0.0  0.0  0.0  ...  0.00  0.0   
13273  0.51  0.0  0.49  0.00  0.0  0.0  0.00  0.0  0.0  0.0  ...  0.00  0.0   
44932  0.00  0.0  0.00  0.00  0.0  0.0  0.00  0.1  0.0  0.0  ...  0.00  0.0   
16990  0.00  0.0  0.00  0.00  0.0  0.0  0.38  0.0  0.0  0.0  ...  0.62  0.0   

         16    17   18   19    20    21   22   23  
51266  0.00  0.00  0.0  0.0  0.55  0.15  0.0  0.0  
42571  0.00  0.00  0.0  0.0  0.00  0.00  0.0  0.0  
13520  0.51  0.49  0.0  0.0  0.00  0.00  0.0  0.0  
43763  0.00  0.00  0.0  0.0  0.00  0.00  0.1  0.0  
44799  0.00  0.10  0.0  0.0  0.00  0.00  0.0  0.0  
...     ...   ...  ...  ...   ...   ...  ...  ...  
34905  0.00  0.00  0.2  0.0  0.00  0.00  0.0  0.0  
28204  0.34  0.33  0.0  0.0  0.00  0.00  0.0  0.0  
13273  0.00  0.00  0.0  0.0  0.00  0.00  0.0  0.0  
44932  0.00  0.00  0.1  0.0  0.00  0.00  0.0  0.0  
16990  0.00  0.00  0.0  0.0  0.00  0.00  0.0  0.0  

[8657 rows x 23 columns]

In [38]:
# Compile the model
def create_model(my_learning_rate):
  """Create and compile a deep neural net."""
  
  # Define the model architecture
  model = tf.keras.models.Sequential()
  model.add(Dense(2000, batch_input_shape=(None,4094),activation='relu'))
  model.add(Dense(1000, batch_input_shape=(None,4094),activation='relu'))
  model.add(Dense(500, batch_input_shape=(None,4094),activation='relu'))
  model.add(Dense(100, batch_input_shape=(None,4094),activation='relu'))  
  model.add(Dense(64, batch_input_shape=(None,4094), activation='relu'))
  model.add(Dense(23, activation=softmax))

  # Compile the model
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=my_learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

  return model 

In [39]:
# Train the model
def train_model(model, train_features, train_label, epochs,
                batch_size=None, validation_split=0.1):
  """Train the model by feeding it data."""

  history = model.fit(x=train_features, y=train_label, batch_size=batch_size,
                      epochs=epochs, shuffle=True, 
                      validation_split=validation_split)
 
  # To track the progression of training, gather a snapshot
  # of the model's metrics at each epoch. 
  epochs = history.epoch
  hist = pd.DataFrame(history.history)

  return epochs, hist    

In [47]:
# The following variables are the hyperparameters.
learning_rate = 0.003
epochs = 100
batch_size = 100
validation_split = 0.2


In [48]:
# Establish the model's topography.
my_model = create_model(learning_rate)

# Train the model on the normalized training set.
epochs, hist = train_model(my_model, X_train, y_train, 
                           epochs, batch_size, validation_split)

Epoch 1/100
277/277 [==============================] - 4s 11ms/step - loss: 252232.5156 - accuracy: 0.1014 - val_loss: 3.1435 - val_accuracy: 0.0475
Epoch 2/100
277/277 [==============================] - 2s 6ms/step - loss: 3.1380 - accuracy: 0.0466 - val_loss: 3.1366 - val_accuracy: 0.0475
Epoch 3/100
277/277 [==============================] - 4s 13ms/step - loss: 3.1359 - accuracy: 0.0445 - val_loss: 3.1359 - val_accuracy: 0.0475
Epoch 4/100
277/277 [==============================] - 2s 7ms/step - loss: 3.1358 - accuracy: 0.0456 - val_loss: 3.1357 - val_accuracy: 0.0472
Epoch 5/100
277/277 [==============================] - 2s 6ms/step - loss: 3.1357 - accuracy: 0.0421 - val_loss: 3.1356 - val_accuracy: 0.0478
Epoch 6/100
277/277 [==============================] - 2s 6ms/step - loss: 3.1358 - accuracy: 0.0399 - val_loss: 3.1356 - val_accuracy: 0.0475
Epoch 7/100
277/277 [==============================] - 1s 5ms/step - loss: 3.1357 - accuracy: 0.0421 - val_loss: 3.1356 - val_accuracy:

In [49]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2000)              8190000   
                                                                 
 dense_1 (Dense)             (None, 1000)              2001000   
                                                                 
 dense_2 (Dense)             (None, 100)               100100    
                                                                 
 dense_3 (Dense)             (None, 64)                6464      
                                                                 
 dense_4 (Dense)             (None, 23)                1495      
                                                                 
Total params: 10,299,059
Trainable params: 10,299,059
Non-trainable params: 0
_________________________________________________________________


In [50]:
from matplotlib import pyplot as plt

In [51]:
#@title Define the plotting function
def plot_curve(epochs, hist, list_of_metrics):
  """Plot a curve of one or more classification metrics vs. epoch."""  
  # list_of_metrics should be one of the names shown in:
  # https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#define_the_model_and_metrics  

  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Value")

  for m in list_of_metrics:
    x = hist[m]
    plt.plot(epochs[1:], x[1:], label=m)

  plt.legend()

print("Loaded the plot_curve function.")

Loaded the plot_curve function.


In [52]:
# Plot a graph of the metric vs. epochs.
list_of_metrics_to_plot = ['accuracy']
plot_curve(epochs, hist, list_of_metrics_to_plot)

In [ ]:
# Evaluate against the test set.
print("\n Evaluate the new model against the test set:")
my_model.evaluate(x=X_test, y=y_test, batch_size=batch_size)

In [35]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test,batch_size=batch_size)


10/10 [==============================] - 0s 6ms/step - loss: 3.1342 - accuracy: 0.0385


In [ ]:
# Make predictions
predictions = model.predict(X_new)

NameError: ignored

In [ ]:
ailab.samk.fi:2291

In [ ]:
gas=rows_gas.index
gas

In [ ]:
df_labels_list.head(-1)

In [ ]:
df_labels_list.sum(axis=1)

In [ ]:
len(labels_list)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# Scale data before applying PCA
scaling=StandardScaler()

# Use fit and transform method
scaling.fit(df_wavelengths)
Scaled_data=scaling.transform(df_wavelengths)


In [ ]:
# Set the n_components=3
principal=PCA(n_components=3)
principal.fit(Scaled_data)
x=principal.transform(Scaled_data)

In [ ]:
# Check the dimensions of data after PCA
print(x.shape)

In [ ]:
# Check the values of eigen vectors
# prodeced by principal components
principal.components_


In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(x[:,0],x[:,1])
plt.xlabel('pc1')
plt.ylabel('pc2')